In [44]:
import pandas as pd
from sqlalchemy import create_engine
import os
from config import pg_password

## Store CSV into DataFrame

In [45]:
movie_fins = os.path.join("Resources","movie_fins.csv")
movie_fins_df = pd.read_csv(movie_fins)
movie_fins_df.head()

,film,revenue,budget,est_profit
0,Inception,825532764,160000000,665532764
1,The Matrix,463517383,63000000,400517383
2,Avengers: Infinity War,2046239637,300000000,1746239637
3,Back to the Future,381109762,19000000,362109762
4,"The Good, the Bad and the Ugly",25118063,1200000,23918063


In [46]:
stream_movies = os.path.join("Resources","MoviesOnStreamingPlatforms_updated.csv")
available_movies_df = pd.read_csv(stream_movies)
available_movies_df.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [55]:
new_available_movies_df = available_movies_df[['Title','Year','Age','IMDb','Rotten Tomatoes','Netflix','Hulu','Prime Video','Disney+','Directors','Genres','Country','Language','Runtime']].copy()
new_available_movies_df.rename(columns={'Title':'title',
                                        'Year': 'year',
                                        'Age':'age',
                                        'IMDb':'imdb',
                                        'Rotten Tomatoes':'rotten_tomatoes',
                                        'Netflix': 'netflix',
                                        'Hulu':'hulu',
                                        'Prime Video':'prime_video',
                                        'Disney+':'disney',
                                        'Directors':'directors',
                                        'Genres':'genres',
                                        'Country':'country',
                                        'Language':'language',
                                        'Runtime':'runtime'}, inplace=True)
new_available_movies_df.head()

,title,year,age,imdb,rotten_tomatoes,netflix,hulu,prime_video,disney,directors,genres,country,language,runtime
0,Inception,2010,13+,8.8,87%,1,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,The Matrix,1999,18+,8.7,87%,1,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,Back to the Future,1985,7+,8.5,96%,1,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


## Connet to local database

In [22]:
pg_user = 'postgres'
# pg_password = 'postgres'
db_name = 'etl_movie_project_db'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

In [23]:
engine.table_names()

[]

## Use Pandas to load csv converted DataFrame into database

In [39]:
movie_fins_df.to_sql(name='movie_financials', con=engine, if_exists='append', index=False)

In [57]:
new_available_movies_df.to_sql(name='movie_streaming', con=engine, if_exists='append', index=False)

## Confirm data has been added by querying something from the table

In [41]:
pd.read_sql_query('select * from movie_financials', con=engine).head()

,film,revenue,budget,est_profit
0,Inception,825532764,160000000,665532764
1,The Matrix,463517383,63000000,400517383
2,Avengers: Infinity War,2046239637,300000000,1746239637
3,Back to the Future,381109762,19000000,362109762
4,"The Good, the Bad and the Ugly",25118063,1200000,23918063


In [58]:
pd.read_sql_query('select * from movie_streaming', con=engine).head()

,title,year,age,imdb,rotten_tomatoes,netflix,hulu,prime_video,disney,directors,genres,country,language,runtime
0,Inception,2010,13+,8.8,87%,1,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,The Matrix,1999,18+,8.7,87%,1,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,Back to the Future,1985,7+,8.5,96%,1,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [ ]:
# Create query and get back a database that will be the final output

# SELECT mf.film, ms.year, ms.netflix, ms.hulu, ms.prime_video, ms.disney, mf.revenue, mf.budget, mf.est_profit
# FROM movie_financials AS mf
# INNER JOIN movie_streaming AS ms ON
# mf.film = ms.title;